In [34]:
import numpy as np # library that supports large and multi-dimensional arrays and matrices
import pandas as pd # library for data manipulation and analysis
import scipy.io # to read data from and write data to a variety of file formats
from scipy import sparse # 2D sparse matrix package
from sklearn.decomposition import PCA # linear dimensionality reduction
from sklearn.linear_model import Ridge # linear least squares with L2 regularization
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from scipy.stats import zscore
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import math
from matplotlib.pyplot import *
import time

In [35]:
class Reservoir(object):
    """
    Build a reservoir and evaluate internal states
    
    Parameters:
        n_internal_units = processing units in the reservoir
        spectral_radius = largest eigenvalue of the reservoir matrix of connection weights
        leak = amount of leakage in the reservoir state update (optional)
        connectivity = percentage of nonzero connection weights (unused in circle reservoir)
        input_scaling = scaling of the input connection weights
        noise_level = deviation of the Gaussian noise injected in the state update
        circle = generate determinisitc reservoir with circle topology
    """
    
    def __init__(self, n_internal_units=100, spectral_radius=0.99, leak=None,
                 connectivity=0.3, input_scaling=0.2, noise_level=0.01, circle=False):
        
        # Initialize attributes
        self._n_internal_units = n_internal_units
        self._input_scaling = input_scaling
        self._noise_level = noise_level
        self._leak = leak

        # Input weights depend on input size: they are set when data is provided
        self._input_weights = None

        # Generate internal weights
        if circle:
            self._internal_weights = self._initialize_internal_weights_Circ(
                    n_internal_units,
                    spectral_radius)
        else:
            self._internal_weights = self._initialize_internal_weights(
                n_internal_units,
                connectivity,
                spectral_radius)


    def _initialize_internal_weights_Circ(self, n_internal_units, spectral_radius):
        
        internal_weights = np.zeros((n_internal_units, n_internal_units))
        internal_weights[0,-1] = spectral_radius
        for i in range(n_internal_units-1):
            internal_weights[i+1,i] = spectral_radius
                
        return internal_weights
    
    
    def _initialize_internal_weights(self, n_internal_units,
                                     connectivity, spectral_radius):

        # Generate sparse, uniformly distributed weights.
        internal_weights = sparse.rand(n_internal_units,
                                       n_internal_units,
                                       density=connectivity).todense()

        # Ensure that the nonzero values are uniformly distributed in [-0.5, 0.5]
        internal_weights[np.where(internal_weights > 0)] -= 0.5
        
        # Adjust the spectral radius.
        E, _ = np.linalg.eig(internal_weights)
        e_max = np.max(np.abs(E))
        internal_weights /= np.abs(e_max)/spectral_radius       

        return internal_weights


    def _compute_state_matrix(self, X, n_drop=0):
        N, T, _ = X.shape
        previous_state = np.zeros((N, self._n_internal_units), dtype=float)

        # Storage
        state_matrix = np.empty((N, T - n_drop, self._n_internal_units), dtype=float)
        for t in range(T):
            current_input = X[:, t, :]

            # Calculate state
            state_before_tanh = self._internal_weights.dot(previous_state.T) + self._input_weights.dot(current_input.T)

            # Add noise
            state_before_tanh += np.random.rand(self._n_internal_units, N)*self._noise_level

            # Apply nonlinearity and leakage (optional)
            if self._leak is None:
                previous_state = np.tanh(state_before_tanh).T
            else:
                previous_state = (1.0 - self._leak)*previous_state + np.tanh(state_before_tanh).T

            # Store everything after the dropout period
            if (t > n_drop - 1):
                state_matrix[:, t - n_drop, :] = previous_state

        return state_matrix


    def get_states(self, X, n_drop=0, bidir=True):
        N, T, V = X.shape
        if self._input_weights is None:
            self._input_weights = (2.0*np.random.binomial(1, 0.5 , [self._n_internal_units, V]) - 1.0)*self._input_scaling

        # compute sequence of reservoir states
        states = self._compute_state_matrix(X, n_drop)
    
        # reservoir states on time reversed input
        if bidir is True:
            X_r = X[:, ::-1, :]
            states_r = self._compute_state_matrix(X_r, n_drop)
            states = np.concatenate((states, states_r), axis=2)

        return states
    
    def getReservoirEmbedding(self, X,pca, ridge_embedding,  n_drop=0, bidir=True, test = False):

        res_states = self.get_states(X, n_drop=0, bidir=True)

        N_samples = res_states.shape[0]
        res_states = res_states.reshape(-1, res_states.shape[2])                    
        # ..transform..
        if test:
            red_states = pca.transform(res_states)
        else:
            red_states = pca.fit_transform(res_states)          
        # ..and put back in tensor form
        red_states = red_states.reshape(N_samples,-1,red_states.shape[1])
        print("red_states:" + str(red_states.shape))

        coeff_tr = []
        biases_tr = []   
        
        for i in range(X.shape[0]):
            ridge_embedding.fit(red_states[i, 0:-1, :], red_states[i, 1:, :])
            coeff_tr.append(ridge_embedding.coef_.ravel())
            biases_tr.append(ridge_embedding.intercept_.ravel())
        #print(np.array(coeff_tr).shape,np.array(biases_tr).shape)
        input_repr = np.concatenate((np.vstack(coeff_tr), np.vstack(biases_tr)), axis=1)
        return input_repr

In [36]:
df_train = pd.read_csv('AWS19_train_64_RIPE.csv', header=None)
df_train.astype(int)
df_test = pd.read_csv('AWS19_test_64_RIPE.csv', header=None)
df_test.astype(int)

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,2330,23,30,0,1606,129,4332,343,5,18,...,0,0,0,0,0,3844,2,524,378,1
1,2331,23,31,0,1626,116,3404,454,5,18,...,0,0,0,0,0,3547,1,546,371,1
2,2332,23,32,0,1426,112,2941,213,5,25,...,0,0,0,0,0,3877,2,465,375,1
3,2333,23,33,0,1871,117,5179,589,6,26,...,0,0,0,0,0,4586,2,534,381,1
4,2334,23,34,0,2205,125,6225,240,6,24,...,0,0,0,0,0,4615,2,808,382,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4345,2355,23,55,0,2500,90,14031,192,5,22,...,0,0,0,0,0,4558,3,788,409,0
4346,2356,23,56,0,1397,129,3703,219,5,24,...,0,0,0,0,0,4290,8,506,385,0
4347,2357,23,57,0,1187,85,2145,138,5,27,...,0,0,0,0,0,2876,4,378,363,0
4348,2358,23,58,0,1589,79,2623,125,5,73,...,0,0,0,0,0,4121,0,544,368,0


In [37]:
def targetify(s):
    if s == 0.0:
        return 0
    else:
        return 1

In [38]:
#df_test = pd.read_csv('slammer_test.csv', header=None)


In [39]:
def eqArray(a,b):
    return np.where(a == b, 1, 0)

In [40]:
cols = [38, 4, 13, 39, 5, 37, 40, 6, 11, 12, 15, 7, 10, 9, 16, 14, 24, 8, 26, 25 ] # checked: correct
features = df_train.columns[cols]


In [41]:
#fraction = 0.5
#print(str(num_features) + " features")
#print("fraction:" + str(fraction))
#data = df_train.sample(frac=fraction, replace=True, random_state=1)

# get X and y. Normalize X and make it into 3D shape for reservoir
# num_col = df_train.shape[1]
# num_row = df_train.shape[0]

df_train['Target']=df_train.loc[:,41].apply(targetify)
df_test['Target']=df_test.loc[:,41].apply(targetify)

X_data = df_train[features]
X_data[features] = X_data[features].apply(pd.to_numeric, errors='coerce', axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_data.values)
X = np.nan_to_num(x_scaled)
if len(X.shape) < 3:
    X = np.atleast_3d(X)
y = df_train['Target']
print("Finished loading training X and y......")

X_data_test = df_test[features]
X_data_test[features] = X_data_test[features].apply(pd.to_numeric, errors='coerce', axis=1)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled_test = min_max_scaler.fit_transform(X_data_test.values)
Xtest = np.nan_to_num(x_scaled_test)
if len(Xtest.shape) < 3:
    Xtest = np.atleast_3d(Xtest)
ytest = df_test['Target']
print("Finished loading test X and y......")

X_train = X
y_train = y
X_test = Xtest
y_test = ytest
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

# split into training and testing data
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=100)

print("X_train shape:" + str(X_train.shape), "y_train shape:" + str(y_train.shape))
print("X_test shape:" + str(X_test.shape), "y_test shape:" + str(y_test.shape))

pca = PCA() #n_components gives number of components to keep for linear dimensionality reduction
ridge_embedding = Ridge(alpha=10, fit_intercept=True)
readout = Ridge(alpha=5)

C:\Users\kdagi\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Finished loading training X and y......
Finished loading test X and y......
(5730, 20, 1) (5730,)
(4350, 20, 1) (4350,)
X_train shape:(5730, 20, 1) y_train shape:(5730,)
X_test shape:(4350, 20, 1) y_test shape:(4350,)


C:\Users\kdagi\Anaconda3\lib\site-packages\pandas\core\frame.py:3494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [42]:
y_train.value_counts()
y_test.value_counts()

0    2880
1    1470
Name: Target, dtype: int64

In [43]:
n=30 #number of internal units
print(str(n) + " internal units")
start = time.clock()
#run through reservoir
res = Reservoir(n_internal_units=n, spectral_radius=0.9, leak=0.2,
     connectivity=0.25, input_scaling=0.3, noise_level=0.01, circle=False)
input_repr = res.getReservoirEmbedding(np.array(X_train), pca, ridge_embedding,  n_drop=0, bidir=False, test = False)
print("Finished loading training reservoir embedding......")
end = time.clock()
input_repr_te = res.getReservoirEmbedding(np.array(X_test), pca, ridge_embedding,  n_drop=0, bidir=False, test = True)
print("Finished loading testing reservoir embedding......")

#fit output
readout.fit(input_repr, y_train)
pred_class = readout.predict(input_repr_te)
#predictions = [int(round(x)) for x in pred_class]
true_class = list(y_test)

#analysis
compdf = pd.DataFrame({'pred_class':pred_class, 'true_class':true_class})
compdf = compdf.sort_values('pred_class', ascending=False)
print(str(compdf.head(10)))

30 internal units
red_states:(5730, 20, 60)
Finished loading training reservoir embedding......
red_states:(4350, 20, 60)
Finished loading testing reservoir embedding......
      pred_class  true_class
540     1.087074           1
2373    1.084801           0
2461    1.076369           0
3900    1.049317           0
331     1.044513           1
2215    1.017385           0
1096    1.013517           1
1718    1.010896           0
2897    0.998020           0
1858    0.995306           0


In [44]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [45]:
predictions = list(compdf['pred_class'].apply(myRound, r=400))
true_class = list(compdf['true_class'])
accuracy = np.sum(list(map(eqArray, predictions, true_class))) / len(true_class)
accuracy

0.6832183908045977

In [46]:
from sklearn.metrics import confusion_matrix

In [47]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [48]:
confm = confusion_matrix(true_class, predictions)
confm

array([[2121,  759],
       [ 619,  851]], dtype=int64)

In [49]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.6832183908045977

In [50]:
print("False alarm rate is")
fp/(tn+fp)

False alarm rate is


0.2635416666666667

In [51]:
print("Precision is")
Precision = tp/(tp+fp)
Precision

Precision is


0.5285714285714286

In [52]:
print("Recall is")
Recall = tp/(tp+fn)
Recall

Recall is


0.5789115646258504

In [53]:
print("F1 is")
2*Precision*Recall/(Precision+Recall)

F1 is


0.5525974025974026

In [54]:
print("Run Time: %.4f" % (end-start))

Run Time: 12.3133
